# Convolutional NN

**Components of the Model**

* Model Function (cnn_model_fn)
    * accept features, class labels, model and model parameters as args 
    * define the layers 
    * define a dictionary for the output of prediction 
    * create `EstimatorSpecs` object for the appropriate mode 
        * train, predict, evaluate 
        * create one-hot from class labeles for train and evaluate 
        * evaluate need dict of metrics to use 
        
* Main Function 
    * accept model and model parameters as args 
    * call a function to get data 
        * load MNIST from TF in this case
    * create the estimator with `cnn_model_fn` and model  parameters 
    * create `*_input_fn` where * is the mode (ex. train)
        * use numpy_input_fn from the TF API for numpy data 
    * run the classifier/estimator using the appropriate mode 
    * perform any work nessesary to display or return result 

In [2]:
import tensorflow as tf
import numpy as np 
import pandas as pd
from pandas import DataFrame as DF, Series 

In [8]:
tf.logging.set_verbosity(tf.logging.INFO) 

## Model Function 
def cnn_model_fn(features, labels, mode, params): 
    """Model function for CNN.""" 
    # Input Layer 
    # Reshape X to 4-D telisor: (batch_size, width, height, channels) 
    # MNIST images are 28x28 pixels, and have one color channel 
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1]) # channels = 1 cause they are blacka and white if they are rgb then it will be 3
    
    # Computes 32 features using a 5x5 filter with ReLU activation 
    # Input Tensor Shape: (batch_size, 28, 28, 1] 
    # Output Tensor Shape: (batch_size, 28, 28, 32]  #it's the same 28, 28 cause we are using padding same , 32 is the filter
    conv1 = tf.layers.conv2d( 
                inputs=input_layer, 
                filters=32, 
                kernel_size=[5, 5], #our filter size
                padding="same", # use "valid" to not preserve WxH 
                activation=tf.nn.relu) 
    
    # First max pooling layer with a 2x2 filter and stride of 2 
    #stride determin if we have an overlap or not we can make it skip as well but thats not a normal thing to do
    #if the stride is the same as the filter width insures that there is no overlap 
    # Input Tensor Shape: (batch_size, 28, 28, 32) 
    # Output Tensor Shape: (batch_size, 14, 14, 32) 
    pooll = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2) 
    
    # Computes 64 feature maps using a 5x5 filter. 
    # Input Tensor Shape: (batch_size, 14, 14, 32) 
    # Output Tensor Shape: (batch_size, 14, 14, 64)
    conv2 = tf.layers.conv2d( 
                inputs=pooll, 
                filters=64, 
                kernel_size=[5, 51], 
                padding="same", 
                activation=tf.nn.relu) 
    
    # Input Tensor Shape: (batch_size, 14, 14, 64) 
    # Output Tensor Shape: (batch_size, 7, 7, 64] 
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2) 

    # Flatten tensor into a batch of vectors for input to dense layer 
    # -1 -> for the batch 
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64]) 
    
    # Input Tensor Shape: (batch_size, 7 * 7 * 64] 
    # Output Tensor Shape: (batch_size, 1024] 
    # units = nodes  
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu) 
    
    # Add dropout operation; 0.6 probability that element will be kept 
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN) 
    
    # Logits layer 
    # Input Tensor Shape: (batch_size, 1024) 
    # Output Tensor Shape: (batch_size, 10)
    logits = tf.layers.dense(inputs=dropout, units=10) 
    
    # this dict will be returned for predictions 
    predictions={ 
        # actual class predictions 
        "classes": tf.argmax(input=logits, axis=1), 
        # class probabilities from softmax on logits 
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor") 
    } 
    
    if mode == tf.estimator.ModeKeys.PREDICT: 
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions) 
    
    # Calculate loss (for both TRAIN and EVAL modes) 
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10) 
    loss = tf.losses.softmax_cross_entropy( onehot_labels=onehot_labels, logits=logits) 
    
    # Configure the training Op (for TRAIN mode) 
    if mode == tf.estimator.ModeKeys.TRAIN: 
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001) 
        train_op = optimizer.minimize( loss=loss, global_step=tf.train.get_global_step()) 
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op) 
    
    # evaluation metrics (for EVAL mode) 
    eval_metric_ops = { 
        "accuracy": tf.metrics.accuracy( 
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec( 
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops) 


## Main Function 
def main(mode='train', model_params={'learning_rate': 0.001}): 
    # Load training and eval data 
    mnist = tf.contrib.learn.datasets.load_dataset("mnist") 
    train_data = mnist.train.images # Returns np.array 
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32) 
    eval_data = mnist.test.images # Returns np.array 
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32) 
    
    #Create the Estimator 
    mnist_classifier = tf.estimator.Estimator( 
        model_fn=cnn_model_fn, params=model_params, model_dir="/tmp/mnist_convnet_model") 
    
    # Train the model 
    if mode == 'train': 
        train_input_fn = tf.estimator.inputs.numpy_input_fn( 
            x={"x": train_data}, 
            y=train_labels, 
            batch_size=100, 
            num_epochs=10, 
            shuffle=True) 
        mnist_classifier.train( 
            input_fn=train_input_fn) 
    elif mode == 'predict': 
        predict_input_fn = tf.estimator.inputs.numpy_input_fn( 
            x={"x": eval_data}, 
            num_epochs=1, 
            shuffle=False) 
        preds = mnist_classifier.predict( 
            input_fn=predict_input_fn) 
        return np.array([p for p in preds]) 
    elif mode == 'eval': 
        #Evaluate the model and print results 
        eval_input_fn = tf.estimator.inputs.numpy_input_fn( 
            x={"x": eval_data}, 
            y=eval_labels, 
            num_epochs=1, 
            shuffle=False) 
        eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn) 
        print(eval_results) 


In [ ]:
main()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000008F8026D828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:te